In [1]:
import numpy as np
import matplotlib.pyplot as plt
import datetime

from proj1_helpers import *
from implementations import *

# constants
training_data = '/Users/karunya/Documents/EPFL/ML/Project_1/train.csv'
test_data = '/Users/karunya/Documents/EPFL/ML/Project_1/test.csv'

#### Helper functions

In [2]:
def error(y, tx, w):
    """Calculates the error in current prediction."""
    return y - np.dot(tx, w)


def compute_loss(y, tx, w):
    """Calculates the loss using MSE."""
    N = y.shape[0]
    e = error(y, tx, w)
    factor = 1/(2*N)
    loss = (np.dot(np.transpose(e), e)) * factor
    return loss


def compute_gradient(y, tx, w):
    """Computes the gradient of the MSE loss function."""
    N = y.shape[0]
    e = error(y, tx, w)
    factor = -1/N
    grad = (np.dot(np.transpose(tx), e)) * factor
    loss = compute_loss(y, tx, w)
    return grad, loss


def compute_stoch_gradient(y, tx, w):
    """Computes a stochastic gradient from a few examples n and their corresponding y_n labels."""
    N = y.shape[0]
    e = error(y, tx, w)
    factor = -1/N
    grad = (np.dot(np.transpose(tx), e)) * factor
    loss = compute_loss(y, tx, w)
    return grad, loss


def sigma(x):
    """Calculates sigma using the formula."""
    return np.exp(x)/(1+np.exp(x))


def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generates a minibatch iterator for a dataset.
    Takes as input two iterables - the output desired values 'y' and the input data 'tx'.
    Outputs an iterator which gives mini-batches of batch_size matching elements from y and tx.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use:
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        do something
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]

            
def compare_prediction(w_train, x, y):
    """Calculates accuracy by comparing prediction with given test data."""
    pred = predict_labels(w_train, x)
    N = len(pred)
    count = 0.0
    for i in range(len(pred)):
        if pred[i] == y[i]:
            count += 1
    # matches = (y == pred).sum()
    return count/N


def split_data(tx, ty, ratio, seed=1):
    """Split training data by ratio"""
    np.random.seed(seed)
    split_idxs = [i for i in range(len(tx))]
    
    # Shuffle the indicies randomly
    np.random.shuffle(split_idxs)
    tx_shuffled = tx[split_idxs]
    ty_shuffled = ty[split_idxs]
    
    # Split by ratio
    split_pos = int(len(tx) * ratio)
    x_train = tx_shuffled[:split_pos]
    x_test = tx_shuffled[split_pos:]
    y_train = ty_shuffled[:split_pos]
    y_test = ty_shuffled[split_pos:]
    
    return x_train, y_train, x_test, y_test 


def build_poly(x, degree):
    """Builds a polynomial of the given degree and appends it to the given matrix."""
    x_ret = x
    for i in range(2, degree+1):
        x_ret = np.c_[x_ret, np.power(x, i)]
    return x_ret


def standardize(x):
    """Standardizes the matrix by calculating the mean of each column and subtracting it from individual values."""
    x_ret = x.copy()
    for i in range(0, x.shape[1]):
        if i==22:
            continue
        x_ret[:,i] -= np.mean(x_ret[:,i])
        x_ret[:,i] /= np.std(x_ret[:,i])
    return x_ret


def replace_999(x):
    x[x == -999.0] = np.nan
    col_mean = np.nanmean(x, axis=0)
    inds = np.where(np.isnan(x))
    x[inds] = np.take(col_mean, inds[1])
    return x


def one_hot_encoding(x):
    col = x[:,22]
    b = np.zeros((x.shape[0], 4))
    for i in range(len(b)):
        a = int(col[i])
        b[i][a] = 1
    x = np.delete(x, 22, 1)
    x = np.hstack((x, b))
    return x

#### Functions to Implement:

In [3]:
def least_squares(y, tx):
    """Calculates the solution using the least squares method."""     
    gram = np.dot(np.transpose(tx),tx)
    gram = np.linalg.inv(gram)
    
    w = np.dot(gram,np.transpose(tx))
    w = np.dot(w, y)
    loss = compute_loss(y, tx, w)
    return w, loss


def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    """Calculates the solution using the least squares with gradient descent method."""
    w = initial_w
    for n_iter in range(max_iters):
        grad, loss = compute_gradient(y, tx ,w)
        w = w - gamma * grad  
    return w, loss


def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    """Calculates the solution using the least squares with stochastic gradient descent method."""
    w = initial_w
    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=1, num_batches=1):
            grad, _ = compute_stoch_gradient(y_batch, tx_batch, w)
            w = w - gamma * grad
            loss = compute_loss(y, tx, w)
    return w, loss


def ridge_regression(y, tx, lambda_):
    """Calculates the solution using the ridge regression method."""
    N = tx.shape[1]
    a = np.dot(np.transpose(tx), tx) + lambda_ * np.identity(N)
    b = np.dot(np.transpose(tx), y)
    w = np.linalg.solve(a, b)
    loss = compute_loss(y, tx, w)
    return w, loss


def logistic_regression(y, tx, initial_w, max_iters, gamma):
    """Calculates the solution using the logistic regression method."""
    w = initial_w
    for n_iter in range(max_iters):
        yx = np.dot(y, tx)
        yxw = np.dot(yx, w)
        log = np.log(1 + np.exp(np.dot(tx, w)))
        loss = (log - yxw).sum()
        
        # Update rule
        sig = sigma(np.dot(tx, w))
        sig = sig - y
        grad = np.dot(np.transpose(tx), sig)
        w = w - gamma * grad 
    return w, loss


def reg_logistic_regression(y, tx, lambda_ , initial_w, max_iters, gamma):
    """Calculates the solution using the regularized logistic regression using gradient descent method."""
    w = initial_w
    for n_iter in range(max_iters):
        yx = np.dot(y, tx)
        yxw = np.dot(yx, w)
        log = np.log(1 + np.exp(np.dot(tx, w)))
        
        # Add the 'penalty' term
        loss = (log - yxw).sum() - (lambda_/2)* np.square((np.linalg.norm(w)))
        
        # Update rule
        sig = sigma(np.dot(tx, w))
        sig = sig - y
        grad = np.dot(np.transpose(tx), sig) + 2 * lambda_*w
        w = w - gamma * grad
    return w, loss

### Retrieve input training and testing data

In [4]:
ty, tx, ids_train = load_csv_data(training_data, sub_sample = False)

fy, fx, ids_test = load_csv_data(test_data, sub_sample = False)

### Preprocess Data

In [5]:
orig_tx = tx.copy()
orig_ty = ty.copy()
orig_fx = fx.copy()

# replace 999's
x_train = replace_999(tx)
fx_train = replace_999(fx)

# normalize data
x_train = standardize(x_train)
fx_train = standardize(fx_train)

# split data by 80-20
x_train, y_train, x_test, y_test = split_data(x_train, ty, 0.80, seed=1)

# hot-encoding
x_train = one_hot_encoding(x_train)
x_test = one_hot_encoding(x_test)
fx_train = one_hot_encoding(fx_train)

# add intercept
x_train = np.hstack((np.ones((x_train.shape[0], 1)), x_train))
x_test = np.hstack((np.ones((x_test.shape[0], 1)), x_test))
fx_train = np.hstack((np.ones((fx_train.shape[0], 1)), fx_train))

### Least Squares Method

In [6]:
w_ls, loss_ls = least_squares(y_train, x_train)
ls_accuracy = compare_prediction(w_ls, x_test, y_test)
print('Accuracy with training \'test set\': ' + str(ls_accuracy * 100) + '%')

ls_accuracy = compare_prediction(w_ls, fx_train, fy)
print('Accuracy with actual test set: ' + str(ls_accuracy * 100) + '%')

Accuracy with training 'test set': 66.482%
Accuracy with actual test set: 65.11496943182257%


### Least Squares using Gradient Descent (using MSE)

In [7]:
max_iters = 10
step_size = 1e-1
w_0 = np.ones(x_train.shape[1])

w_lsgd, loss_lsgd = least_squares_GD(y_train, x_train, w_0, max_iters, step_size)
lsGD_accuracy = compare_prediction(w_lsgd, x_test, y_test)
print('Accuracy with training \'test set\': ' + str(lsGD_accuracy * 100) + '%')

lsGD_accuracy = compare_prediction(w_lsgd, fx_train, fy)
print('Accuracy with actual test set: ' + str(lsGD_accuracy * 100) + '%')

Accuracy with training 'test set': 53.080000000000005%
Accuracy with actual test set: 56.79380822824239%


### Least Squares using Stochastic Gradient Descent (batch_size = 1)

In [8]:
max_iters = 10
step_size = 1e-1
w_0 = np.zeros(x_train.shape[1])

w_lstoch, loss_lstoch = least_squares_SGD(y_train, x_train, w_0, max_iters, step_size) 
lsStoch_accuracy = compare_prediction(w_lstoch, x_test, y_test)
print('Accuracy with training \'test set\': '  + str(lsStoch_accuracy * 100) + '%')

lsStoch_accuracy = compare_prediction(w_lstoch, fx_train, fy)
print('Accuracy with actual test set: ' + str(lsStoch_accuracy * 100) + '%')

Accuracy with training 'test set': 33.995999999999995%
Accuracy with actual test set: 65.88471731915149%


### Ridge Regression

In [9]:
# variables
degree = 12
lambda_ = 0.15256

# replace 999's
x_train = replace_999(tx)
fx_train = replace_999(fx)
# print('Replacing 999: ' + str(x_train.shape))

# normalize data
x_train = standardize(x_train)
fx_train = standardize(fx_train)
# print('Normalize: ' + str(x_train.shape))

# split data by 80-20
x_train, y_train, x_test, y_test = split_data(x_train, ty, 0.75, seed=1)

# hot-encoding
x_train = one_hot_encoding(x_train)
x_test = one_hot_encoding(x_test)
fx_train = one_hot_encoding(fx_train)
# print('One-hot Encoding: ' + str(x_train.shape))

# polynomial-fit
x_train = build_poly(x_train, degree)
x_test = build_poly(x_test, degree)
fx_train = build_poly(fx_train, degree)
# print('Build Poly: ' + str(x_train.shape))

# add intercept
x_train = np.hstack((np.ones((x_train.shape[0], 1)), x_train))
x_test = np.hstack((np.ones((x_test.shape[0], 1)), x_test))
fx_train = np.hstack((np.ones((fx_train.shape[0], 1)), fx_train))
# print('Add intercept: ' + str(x_train.shape))

w_rr, loss_rr = ridge_regression(y_train, x_train, lambda_)
rr_accuracy = compare_prediction(w_rr, x_test, y_test)

print('Degree: ' + str(degree))
print('Lambda: ' + str(lambda_))
print('Accuracy with training \'test set\': '  + str(rr_accuracy * 100) + '%')

rr_accuracy = compare_prediction(w_rr, fx_train, fy)
print('Accuracy with test set: '  + str(rr_accuracy * 100) + '%')

Degree: 12
Lambda: 0.15256
Accuracy with training 'test set': 81.7808%
Accuracy with test set: 30.571521087994817%


### Logistic Ridge Regression

In [10]:
# variables
max_iters = 100
step_size = 3

# replace 999's
x_train = replace_999(orig_tx)
fx_train = replace_999(orig_fx)

# normalize data
# x_train = standardize(x_train)
# fx_train = standardize(fx_train)

# split data by 80-20
x_train, y_train, x_test, y_test = split_data(x_train, ty, 0.75, seed=1)

# hot-encoding
# x_train = one_hot_encoding(x_train)
# x_test = one_hot_encoding(x_test)
# fx_train = one_hot_encoding(fx_train)

# add intercept
x_train = np.hstack((np.ones((x_train.shape[0], 1)), x_train))
x_test = np.hstack((np.ones((x_test.shape[0], 1)), x_test))
fx_train = np.hstack((np.ones((fx_train.shape[0], 1)), fx_train))

w_0 = np.zeros(x_train.shape[1])

w_lrr, loss_rr = logistic_regression(y_train, x_train, w_0, max_iters, step_size)

lrr_accuracy = compare_prediction(w_lrr, x_test, y_test)
print('Accuracy with training \'test set\': '  + str(lrr_accuracy * 100) + '%')

lrr_accuracy = compare_prediction(w_lrr, fx_train, fy)
print('Accuracy with test set: '  + str(lrr_accuracy * 100) + '%')

Accuracy with training 'test set': 65.5808%
Accuracy with test set: 0.0%


### Regularized Logistic Regression

In [11]:
# variables
lambda_ = 0.15256
max_iters = 10
step_size = 0.003

# replace 999's
x_train = replace_999(orig_tx)
fx_train = replace_999(orig_fx)

# # normalize data
# x_train = standardize(x_train)
# fx_train = standardize(fx_train)

# split data by 80-20
x_train, y_train, x_test, y_test = split_data(x_train, ty, 0.75, seed=1)

# hot-encoding
x_train = one_hot_encoding(x_train)
x_test = one_hot_encoding(x_test)
fx_train = one_hot_encoding(fx_train)

# add intercept
x_train = np.hstack((np.ones((x_train.shape[0], 1)), x_train))
x_test = np.hstack((np.ones((x_test.shape[0], 1)), x_test))
fx_train = np.hstack((np.ones((fx_train.shape[0], 1)), fx_train))

w_0 = np.zeros(x_train.shape[1])

w_rlr, loss_rr = reg_logistic_regression(y_train, x_train, lambda_ , w_0, max_iters, step_size)

rlr_accuracy = compare_prediction(w_rlr, x_test, y_test)
print('Accuracy with training \'test set\': '  + str(lrr_accuracy * 100) + '%')

rlr_accuracy = compare_prediction(w_rlr, fx_train, fy)
print('Accuracy with test set: '  + str(rlr_accuracy * 100) + '%')

Accuracy with training 'test set': 0.0%
Accuracy with test set: 0.0%


### Write Predictions to Output File

In [12]:
# y_pred = predict_labels(w_rr, fx_train)
# print('Predictions shape: ' + str(y_pred.shape))
# create_csv_submission(ids_test, y_pred, "output.csv")